# AMPLPY: Using Google Sheets

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplpy/blob/master/notebooks/gspread.ipynb)

Documentation: http://amplpy.readthedocs.io

GitHub Repository: https://github.com/ampl/amplpy

PyPI Repository: https://pypi.python.org/pypi/amplpy

Jupyter Notebooks: https://github.com/ampl/amplpy/tree/master/notebooks

- This notebook uses the snippet from https://colab.research.google.com/notebooks/snippets/sheets.ipynb in order to load data from the Goolge Sheet at https://docs.google.com/spreadsheets/d/1sTyJdgnMCrmuZDtUjs-cOpRLoKgByM8U-lHieNBNaRY/edit?usp=sharing

### Autheticate in order to use Google Sheets

In [1]:
from google.colab import auth
auth.authenticate_user()

### Setup

In [2]:
!pip install -q amplpy ampltools gspread --upgrade

     |████████████████████████████████| 4.1 MB 4.4 MB/s 


### Google Colab & Kaggle interagration

In [3]:
MODULES=['ampl', 'coin']
from ampltools import cloud_platform_name, ampl_notebook
from amplpy import AMPL, register_magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = ampl_notebook(modules=MODULES) # Install AMPL and use it
register_magics(ampl_object=ampl) # Evaluate %%ampl_eval cells with ampl.eval()

Downloading: https://portal.ampl.com/dl/modules/ampl-module.linux64.tgz
> /content/ampl.linux-intel64/README (new)
> /content/ampl.linux-intel64/amplkey (new)
> /content/ampl.linux-intel64/amplxl.dll (new)
> /content/ampl.linux-intel64/ampl.env (new)
> /content/ampl.linux-intel64/models (new)
> /content/ampl.linux-intel64/ampl.lic (new)
> /content/ampl.linux-intel64/docs (new)
> /content/ampl.linux-intel64/ampl (new)
> /content/ampl.linux-intel64/leasefingerprint (new)
> /content/ampl.linux-intel64/fingerprint (new)
Downloading: https://portal.ampl.com/dl/modules/coin-module.linux64.tgz
> /content/ampl.linux-intel64/ipopt (new)
> /content/ampl.linux-intel64/coin-license.txt (new)
> /content/ampl.linux-intel64/bonmin (new)
> /content/ampl.linux-intel64/couenne (new)
> /content/ampl.linux-intel64/coin-versions.txt (new)
> /content/ampl.linux-intel64/cbc (new)
AMPL License:


### Use `%%ampl_eval` to evaluate AMPL commands

In [4]:
%%ampl_eval
option version;

option version 'AMPL Version 20220219 (Linux-5.4.0-1069-azure, 64-bit)\
Demo license with maintenance expiring 20240131.\
Using license file "/content/ampl.linux-intel64/ampl.lic".\
';


### Define the model

In [5]:
%%ampl_eval
set NUTR;
set FOOD;

param cost {FOOD} > 0;
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];

param n_min {NUTR} >= 0;
param n_max {i in NUTR} >= n_min[i];

param amt {NUTR,FOOD} >= 0;

var Buy {j in FOOD} >= f_min[j], <= f_max[j];

minimize Total_Cost:  sum {j in FOOD} cost[j] * Buy[j];

subject to Diet {i in NUTR}:
   n_min[i] <= sum {j in FOOD} amt[i,j] * Buy[j] <= n_max[i];

### Instatiate gspread client

In [6]:
import gspread
from google.auth import default
creds, _ = default()
gclient = gspread.authorize(creds)

def open_spreedsheet(name):
    if name.startswith('https://'):
        return gclient.open_by_url(name)
    return gclient.open(name)

### Open speedsheet using name or URL

In [7]:
# spreedsheet = open_spreedsheet('DietModelSheet')
spreedsheet = open_spreedsheet('https://docs.google.com/spreadsheets/d/1sTyJdgnMCrmuZDtUjs-cOpRLoKgByM8U-lHieNBNaRY/edit?usp=sharing')

def get_worksheet_values(name):
    return spreedsheet.worksheet(name).get_values(value_render_option='UNFORMATTED_VALUE')

### Define auxiliar functions to convert data from worksheets into dataframes

In [8]:
import pandas as pd

def table_to_dataframe(rows):
    return pd.DataFrame(rows[1:], columns=rows[0]).set_index(rows[0][0])

def matrix_to_dataframe(rows, tr=False):
    col_labels = rows[0][1:]
    row_labels = [row[0] for row in rows[1:]]
    def label(pair):
        return pair if not tr else (pair[1], pair[0])
    data = {
        label((rlabel, clabel)): rows[i+1][j+1]
        for i, rlabel in enumerate(row_labels)
        for j, clabel in enumerate(col_labels)}
    df = pd.Series(data).reset_index()
    df.columns = ['index1', 'index2', rows[0][0]]
    return df.set_index(['index1', 'index2'])

### Load data from the first worksheet

In [9]:
rows = get_worksheet_values('FOOD')
df = table_to_dataframe(rows)
ampl.set_data(df, set_name='FOOD')  # send the data to AMPL 
df

,cost,f_min,f_max
FOOD,,,
BEEF,3.19,0,100
CHK,2.59,0,100
FISH,2.29,0,100
HAM,2.89,0,100
MCH,1.89,0,100
MTL,1.99,0,100
SPG,1.99,0,100
TUR,2.49,0,100


### Load the data from the second worksheet

In [10]:
rows = get_worksheet_values('NUTR')
df = table_to_dataframe(rows)
ampl.set_data(df, set_name='NUTR')  # Send the data to AMPL
df

,n_min,n_max
NUTR,,
A,700,10000
C,700,10000
B1,700,10000
B2,700,10000


### Load the data from the third worksheet

In [11]:
rows = get_worksheet_values('amt')
df = matrix_to_dataframe(rows, tr=True)
ampl.set_data(df)  # Send the data to AMPL
df

,,amt
index1,index2,
A,BEEF,60
C,BEEF,20
B1,BEEF,10
B2,BEEF,15
A,CHK,8
C,CHK,0
B1,CHK,20
B2,CHK,20
A,FISH,8


### Use `%%ampl_eval` to solve the model with cbc

In [12]:
%%ampl_eval
option solver cbc;
solve;
display Buy;

CBC 2.10.5: CBC 2.10.5 optimal, objective 88.2
1 iterations
Buy [*] :=
BEEF   0
 CHK   0
FISH   0
 HAM   0
 MCH  46.6667
 MTL   0
 SPG   0
 TUR   0
;



### Retrieve the solution as a pandas dataframe

In [13]:
ampl.var['Buy'].get_values().to_pandas()

,Buy.val
BEEF,0.000000
CHK,0.000000
FISH,0.000000
HAM,0.000000
MCH,46.666667
MTL,0.000000
SPG,0.000000
TUR,0.000000
